# ANEXO 03

# Procesado de los datos

La idea de este código es tener los datos preparados para el análisis. Hemos juntado las variables que se solapan y hemos añadido un ID para ellas. También hemos cambiado el formato de la fecha.
El script devuelve el df completo y un conjunto de prueba para que podamos practicar en otro cuaderno jupyter con ese conjunto de datos.

In [1]:
# General
import pandas as pd
import numpy as np
#---------------------------------------------------------

# Gráficos
import matplotlib.pyplot as plt
#---------------------------------------------------------

# Preprocesado
from datetime import datetime
from sklearn import preprocessing
#-----------------------------------------------------

# Eliminamos los warnings del notebook
import warnings
warnings.filterwarnings('ignore')

## Carga del dataset

In [2]:
df = pd.read_csv('../dataframes/Actividad_comercial_v01.csv')
# print(len(df.index))
# df.head()

## Limpieza de datos

Reemplazamos los valores '99' que indican valores perdidos en la variable 'ID_AGRUP_CANAL column' por NA. Hacemos lo mismo con los valores 'PDTE' (pendiente) de la variable 'ID_ZONA'

In [3]:
df['ID_AGRUP_CANAL'] = df['ID_AGRUP_CANAL'].replace(99, np.nan)

df['ID_ZONA'] = df['ID_ZONA'].replace('PDTE', np.nan)

Consideramos junto con negocio que la mejor idea en esta caso es borrar los NA en lugar de imputarlos, dado que al ser variables categóricas y ser los escenarios independientes las predicciones podrian enturbiar los datos.

In [4]:
df = df.dropna()

Por último, convertimos la columna  de canal en enteros, para evitarnos problemas futuros.

In [5]:
df['ID_AGRUP_CANAL'] = df['ID_AGRUP_CANAL'].astype(int)

## Preparación de los datos para la serie temporal

Vamos convertir los datos mensuales en semanales. Según naturgy podemos dividir los contratos y las cancelaciones eventualmente entre las semanas de un mes, porque para ellos esto tiene sentido teórico y no tienen datos recogidos con esta frecuencia. Cada mes tiene su número específico de semanas dependiendo del año. Tenemos un documento con el número de ellas.

In [6]:
# Cargando los datos
df_weeks = pd.read_csv('../dataframes/data_semanas_2019_2022.csv', sep = ';')


### Crear columna de semanas por mes

Vamos a crear un objeto que contenga el número de semanas por cada mes para cada año, y lo vamos a añadir a nuestro dataframe.

In [7]:
# Número de semanas por mes
num_month_weeks = df_weeks['ID_MES'].value_counts().sort_index().to_dict()

# Transformamos la variable 'ID_MES' del data frame en su índice
df = df.set_index('ID_MES')

# Ahora tenemos una columna con el numero de semanas que tiene cada mes en el dataframe
df['num_month_weeks'] = pd.Series(num_month_weeks)

# Reiniciamos el indice
df = df.reset_index()

df.head(1)

,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,Año,Fecha_Mes,Altas M,Altas_Mes_C+T,Altas_Mes_Tra,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks
0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,2019,31/12/19 00:00,4,4,0,-1,-1,0,3,131,4


### Repartir los datos 'objetivo' entre las semanas

Esto es un primer paso para obtener los datos por semana. Según el product owner de naturgy la distribución de los contratos se puede asumir constante a lo largo de las semanas, por lo que si el mes tiene 4 o 5 semanas el valor mensual se dividirá en consecuencia. Este metodo no se puede aplicar la variable cartera, que se tratará mas adelante.

In [8]:
df['Altas M'] /= df['num_month_weeks']
df['Altas_Mes_C+T'] /= df['num_month_weeks']
df['Altas_Mes_Tra'] /= df['num_month_weeks']
df['Bajas M'] /= df['num_month_weeks']
df['Bajas_Mes_C+T'] /= df['num_month_weeks']
df['Bajas_Mes_Tra'] /= df['num_month_weeks']
df['Neto M'] /= df['num_month_weeks']

In [9]:
df.head(1)

,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,Año,Fecha_Mes,Altas M,Altas_Mes_C+T,Altas_Mes_Tra,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks
0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,2019,31/12/19 00:00,1.0,1.0,0.0,-0.25,-0.25,0.0,0.75,131,4


### Multiplicación de las filas por el numero de semanas del mes

Creamos una funcion que multiplica cada fila por el valor de la columna creada 'num_month_week' y además, numere cada una de las semanas

In [10]:
df_semanas = df.assign(num_month_weeks = df.num_month_weeks.apply(lambda x: range(1, x + 1))).explode('num_month_weeks')
df_semanas = df_semanas.reset_index()

df_semanas.head(4)

,index,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,...,Fecha_Mes,Altas M,Altas_Mes_C+T,Altas_Mes_Tra,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks
0,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,31/12/19 00:00,1.0,1.0,0.0,-0.25,-0.25,0.0,0.75,131,1
1,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,31/12/19 00:00,1.0,1.0,0.0,-0.25,-0.25,0.0,0.75,131,2
2,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,31/12/19 00:00,1.0,1.0,0.0,-0.25,-0.25,0.0,0.75,131,3
3,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,31/12/19 00:00,1.0,1.0,0.0,-0.25,-0.25,0.0,0.75,131,4


### Creación de la variable ID_SEMANA

Necesitamos una variable de ID que idintifique cada semana, para ello queremos unaavariable que tenga el año y en numero de semana, por ejemplo '202037'

In [11]:

# Crear una lista de todos los números referidos a las semanas para obtener los datos de las semanas ordenados para aplicarlos al modelo
semana_año = [[49,50,51,52],['01','02','03','04','05','06','07','08','09'],list(range(10,54)),['01','02','03','04','05','06','07','08','09'], list(range(10,53)),['01','02','03','04','05','06','07','08','09'], list(range(10,28))]
semana_año = [item for sublist in semana_año for item in sublist]


# Construir un diccionario de semanas para ordenar los datos almacenados en una columna llamada ID_SEMANAS
df_semanas['num_week'] = (df_semanas['ID_MES'].apply(str) + '_' + df_semanas['num_month_weeks'].apply(str))
dict_semana = dict(zip(df_semanas['num_week'].unique(), semana_año))
df_semanas.replace({'num_week':dict_semana}, inplace=True) 

#Creamos la columna
df_semanas['ID_SEMANA'] = (df_semanas['Año'].apply(str) + df_semanas['num_week'].apply(str))

# Construir columna date para poder aplicar modificaciones mas adelante
date = dict(zip(df_semanas['ID_SEMANA'].unique(),list(range(1,135))))
df_semanas['date'] = df_semanas['ID_SEMANA']
df_semanas = df_semanas.replace({'date': date})

df_semanas.head(4)

,index,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,...,Altas_Mes_Tra,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks,num_week,ID_SEMANA,date
0,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,0.0,-0.25,-0.25,0.0,0.75,131,1,49,201949,1
1,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,0.0,-0.25,-0.25,0.0,0.75,131,2,50,201950,2
2,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,0.0,-0.25,-0.25,0.0,0.75,131,3,51,201951,3
3,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,0.0,-0.25,-0.25,0.0,0.75,131,4,52,201952,4


### Creación de la variable fecha_semana

A partir de ID_SEMANA creamos una varible en formato fecha para poder usarla en el model

In [12]:
# Creamos un diccionario con el ID de la semana y su fecha asociada.
fecha_dict = dict(zip(df_weeks.ID_SEMANA.apply(str), df_weeks.DES_SEMANA_FECHA))

#Creamos la columna con las fecha con formato string
df_semanas['fecha_semanas'] = df_semanas.ID_SEMANA
df_semanas.replace({'fecha_semanas':fecha_dict}, inplace=True) 

# Damos formato datetime a esa columna
startEndDate = [x.split('-') for x in df_semanas.fecha_semanas]
df_semanas.fecha_semanas = [f[0] for f in startEndDate]
df_semanas['fecha_semanas'] = pd.to_datetime(df_semanas['fecha_semanas'], format='%d/%m/%Y')

df_semanas.head(4)

,index,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,...,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks,num_week,ID_SEMANA,date,fecha_semanas
0,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131,1,49,201949,1,2019-11-30
1,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131,2,50,201950,2,2019-12-07
2,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131,3,51,201951,3,2019-12-14
3,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131,4,52,201952,4,2019-12-21


# Filtro de escenarios

El objetivo es poder aplicar cualquier función que se haga a todos los escenarios posibles en los datos

In [13]:
# Almacenamos el dataframe el otro objeto para poder trabajar sin sobreescribir sobre el anterior.
df = df_semanas.copy()

Queremos crear una lista con todas las combinaciones de escenarios posibles, sin embargo, exiten combinaciones duplicadas, por lo que debemos almaacenar las combinaciones únicas

In [14]:
# Existen duplciadas?
any(df[['ID_AGRUP_PROD','ID_AGRUP_CANAL','ID_ZONA','ID_AGRUP_PRODUCTO','ID_LINEA_PRODUCTO_AGREG','ID_SOCIEDAD','ID_SEGMENTO','SEGMENTO2','ID_MES']].duplicated())

True

In [15]:
# Almacenados las posibles combinaciones unicas
combinaciones_totales = df['ID_SOCIEDAD'] + '_' + df['ID_SEGMENTO'] + '_' + df['SEGMENTO2'] + '_LINEAPRODUCTOAGREG:' + df['ID_LINEA_PRODUCTO_AGREG'].apply(str) + '_AGRUPPRODUCTO:' + df['ID_AGRUP_PRODUCTO'].apply(str) + '_' + df['ID_AGRUP_PROD'] + '_CANAL:' + df['ID_AGRUP_CANAL'].apply(str) + '_' + df['ID_ZONA']
posibles_combinaciones = combinaciones_totales.unique()
len(posibles_combinaciones)

1046

A continuación creamos un bucle que cuenta cuantos datos tenemos de cada escenario, aquellos escenarios que esten desde el inicio tendran 135 datos.

In [16]:
nfilasXcombinacion = pd.DataFrame()

for combinacion in posibles_combinaciones:

    Var_split = str(combinacion).split('_')
    
    ID_SOCIEDAD = Var_split[0]
    ID_SEGMENTO = Var_split[1]
    SEGMENTO2 = Var_split[2]
    ID_LINEA_PRODUCTO_AGREG = Var_split[3].split(':')[1]
    ID_AGRUP_PRODUCTO = Var_split[4].split(':')[1]
    ID_AGRUP_PROD = Var_split[5]
    ID_AGRUP_CANAL = Var_split[6].split(':')[1]
    ID_ZONA = Var_split[7]

    nfilas = len(df[ (df.ID_SOCIEDAD == ID_SOCIEDAD) & 
    (df.ID_SEGMENTO == ID_SEGMENTO) &
    (df.SEGMENTO2 == SEGMENTO2) & 
    (df.ID_LINEA_PRODUCTO_AGREG == float(ID_LINEA_PRODUCTO_AGREG)) & 
    (df.ID_AGRUP_PRODUCTO == float(ID_AGRUP_PRODUCTO)) &
    (df.ID_AGRUP_PROD == ID_AGRUP_PROD) &
    (df.ID_AGRUP_CANAL == float(ID_AGRUP_CANAL)) &
    (df.ID_ZONA == ID_ZONA)])

    nfilasXcombinacion = nfilasXcombinacion.append(
        {
            'combinacion': combinacion,
            'nfilas': nfilas
        }, ignore_index=True)

nfilasXcombinacion.head(5)

,combinacion,nfilas
0,SOC01_SEG01_LP01_LINEAPRODUCTOAGREG:1_AGRUPPRO...,135
1,SOC02_SEG01_LP01_LINEAPRODUCTOAGREG:1_AGRUPPRO...,135
2,SOC02_SEG01_LP01_LINEAPRODUCTOAGREG:1_AGRUPPRO...,135
3,SOC02_SEG02_LP01_LINEAPRODUCTOAGREG:1_AGRUPPRO...,135
4,SOC02_SEG02_LP01_LINEAPRODUCTOAGREG:1_AGRUPPRO...,135


Creamos un filtro que elimina aquellas combinaciones con menos de 50 datos, esto debido a los problemas de rendimiento que se aprecian en series temporales con menos de 50-60 registros (Jeb, 2015; McCleary, 1980; Warner, 1998)

In [17]:
nfilasXcombinacion_selected = nfilasXcombinacion[nfilasXcombinacion.nfilas > 50]['combinacion'].tolist()

### Guardamos estos datos en el entorno para que puedan ser utilizados en un futuro

In [18]:
%store posibles_combinaciones
%store nfilasXcombinacion
%store nfilasXcombinacion_selected

Stored 'posibles_combinaciones' (ndarray)
Stored 'nfilasXcombinacion' (DataFrame)
Stored 'nfilasXcombinacion_selected' (list)


## Cambio de la variable cartera a formato 

In [19]:
# Transformamos df_semanas.date a formato float para reducir el tiempo de ejecucion del bucle hasta 5 veces menos en tiempo.
df_semanas.date = df_semanas.date.astype(float)

In [20]:
for combinacion in nfilasXcombinacion_selected:

    Var_split = str(combinacion).split('_')
        
    ID_SOCIEDAD = Var_split[0]
    ID_SEGMENTO = Var_split[1]
    SEGMENTO2 = Var_split[2]
    ID_LINEA_PRODUCTO_AGREG = float(Var_split[3].split(':')[1])
    ID_AGRUP_PRODUCTO = float(Var_split[4].split(':')[1])
    ID_AGRUP_PROD = Var_split[5]
    ID_AGRUP_CANAL = float(Var_split[6].split(':')[1])
    ID_ZONA = Var_split[7]

    temp_df = df_semanas[(df_semanas.ID_SOCIEDAD == ID_SOCIEDAD) & 
                                        (df_semanas.ID_SEGMENTO == ID_SEGMENTO) & 
                                        (df_semanas.SEGMENTO2 == SEGMENTO2) &
                                        (df_semanas.ID_LINEA_PRODUCTO_AGREG == ID_LINEA_PRODUCTO_AGREG) & 
                                        (df_semanas.ID_AGRUP_PRODUCTO == ID_AGRUP_PRODUCTO) &
                                        (df_semanas.ID_AGRUP_PROD == ID_AGRUP_PROD) & 
                                        (df_semanas.ID_AGRUP_CANAL == ID_AGRUP_CANAL) & 
                                        (df_semanas.ID_ZONA == ID_ZONA)]

    temp_df.sort_values('date')['Cartera_Acu'].iloc[1:] = np.NaN

    for week in range(1, len(temp_df['date'])):
            if week != 1.0:
                temp_df['Cartera_Acu'][df_semanas.date == week] = sum(temp_df['Cartera_Acu'][(temp_df.date == week-1)], temp_df['Neto M'][(temp_df.date == week)])
        
    df_semanas['Cartera_Acu'][(df_semanas.ID_SOCIEDAD == ID_SOCIEDAD) & 
                                        (df_semanas.ID_SEGMENTO == ID_SEGMENTO) & 
                                        (df_semanas.SEGMENTO2 == SEGMENTO2) &
                                        (df_semanas.ID_LINEA_PRODUCTO_AGREG == ID_LINEA_PRODUCTO_AGREG) & 
                                        (df_semanas.ID_AGRUP_PRODUCTO == ID_AGRUP_PRODUCTO) &
                                        (df_semanas.ID_AGRUP_PROD == ID_AGRUP_PROD) & 
                                        (df_semanas.ID_AGRUP_CANAL == ID_AGRUP_CANAL) & 
                                        (df_semanas.ID_ZONA == ID_ZONA)] = temp_df['Cartera_Acu']

### Guardamos el dataset en el entorno virtual con valores de cartera acu actualizados

In [21]:
df_temporal_series = df_semanas.copy()
%store df_temporal_series

df_temporal_series.head(4)

Stored 'df_temporal_series' (DataFrame)


,index,ID_MES,ID_SOCIEDAD,ID_SEGMENTO,SEGMENTO2,ID_LINEA_PRODUCTO_AGREG,ID_AGRUP_PRODUCTO,ID_AGRUP_PROD,ID_AGRUP_CANAL,ID_ZONA,...,Bajas M,Bajas_Mes_C+T,Bajas_Mes_Tra,Neto M,Cartera_Acu,num_month_weeks,num_week,ID_SEMANA,date,fecha_semanas
0,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131.00,1,49,201949,1.0,2019-11-30
1,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,131.75,2,50,201950,2.0,2019-12-07
2,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,132.50,3,51,201951,3.0,2019-12-14
3,0,201912,SOC01,SEG01,LP01,1,1,PROD12,1,ZONA01,...,-0.25,-0.25,0.0,0.75,133.25,4,52,201952,4.0,2019-12-21
